# Worksheet 05

Name: Sumatra K Dhimoyee 
UID: U26967910

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

The first round will give us two clusters (0, 0.5) and (1.5, 2, 6, 6.5, 7) with new centroids at 0.25 and 4.6
The second round will give us two clusters (0,0.5, 1.5, 2) and (6, 6.5, 7) with new centroids at 1 and 6.5. 
The algorithm has converged by this point, and the centroids won't change with new rounds.

b) Describe in plain english what the cost function for k means is.

It calculates the distance between each data point and the centroid in that specific cluster. It tries to minimize the distance to find compact clusters with the closest data points.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

If the data points are densely populated in one region and contain outliers, the initial centroid choice will greatly influence which points the clusters converge. The k-means technique can yield various results for the same number of clusters on a given dataset depending on its initialization, the distribution of data points, and the existence of outliers.

d) Does Lloyd's Algorithm always converge? Why / why not?

No. The algorithm will struggle to converge if clusters form complex shapes such as spirals, if the data points have different densities or sizes across clusters, if the data contain many outliers, or if there is no proper initialization of centroids.

e) Follow along in class the implementation of Kmeans

In [1]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))
        
    def is_unassigned(self, i):
        return self.assignment[i] == -1
    
    def unassign_all(self):
        self.assignment = [-1 for _ in range(len(self.data))]
        
    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), size=self.k, replace=False)]
    
    def are_centers_diff(self, c1, c2):
        for i in range(len(c1)):
            if c1[i] not in c2:
                return True
        return False
    
    def assign(self, centers):
        for i in range(len(self.data)):
            self.assignment[i] = 0
            temp_assign = 0
            temp_dist = self.dist(self.data[i], centers[0])
            for j in range(1, len(centers)):
                new_dist = self.dist(self.data[i], centers[j])
                if temp_dist > new_dist:
                    self.assignment[i] = j
                    temp_dist = new_dist
    
    def calculate_new_centers(self):
        centers = []
        for j in range(self.k):
            cluster_j = self.data[
                np.array([i for i in range(len(self.data)) if self.assignment[i] == j])
            ]
            centers.append(np.mean(cluster_j,axis=0))
        
        return np.array(centers)

    def dist(self, x, y):
        return sum((x - y) ** 2) ** (1/2)

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.calculate_new_centers()
        while self.are_centers_diff(centers, new_centers):
            centers = new_centers
            self.snap(centers)
            self.unassign_all()
            self.assign(centers)
            new_centers = self.calculate_new_centers()
            print (new_centers)
        return
            
kmeans = KMeans(X, 4)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)

[[ 1.83329012 -4.16630999]
 [ 1.25724333  0.92846853]
 [-2.98336083  1.37875497]
 [-1.3506666   2.87587061]]
[[ 1.80375063 -4.10800509]
 [ 1.24539452  1.06153028]
 [-3.10155532  1.54141092]
 [-1.32339914  2.47699442]]
[[ 1.80582124 -4.04080901]
 [ 1.28077416  1.10997533]
 [-3.28860394  1.54673043]
 [-1.39212997  2.02399162]]
[[ 1.77224473 -4.0111356 ]
 [ 1.38476756  1.18521237]
 [-3.48051651  1.66798646]
 [-1.38201497  1.53177849]]
[[ 1.78863971 -4.0086992 ]
 [ 1.56378546  1.35070233]
 [-3.57705858  1.78431564]
 [-1.27843678  1.04915322]]
[[ 1.82416454 -4.00058334]
 [ 1.67091906  1.54426338]
 [-3.49182514  1.87149177]
 [-1.0596207   0.66243957]]
[[ 1.84180743 -4.03258775]
 [ 1.77704065  1.66778766]
 [-3.2608491   1.91559847]
 [-0.66166866  0.29130895]]
[[ 1.84180743 -4.03258775]
 [ 1.86471673  1.81986805]
 [-3.1319776   1.87800012]
 [-0.33425114  0.12218462]]
[[ 1.84126343 -4.06791444]
 [ 1.90165131  1.93255625]
 [-3.07703402  1.85266958]
 [-0.11632593  0.03996848]]
[[ 1.84585117 -4.10